In [ ]:
!pip install llama_index

In [ ]:
!pip install transformers accelerate bitsandbytes

## Setup

### Data

In [3]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 5.5 MB/s eta 0:00:00


In [4]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.16.0-py3-none-any.whl size=273811 sha256=bf64111e2adc90526234ce8a6985ebd499b1042cdac352686ae36dd44304e405
  Stored in directory: /root/.cache/pip/wheels/20/a0/f6/87a1ec9636c915fe2d6c6e859fd55a6231dd9bc95a1d5394b1
Successfully built neo4j


In [5]:
from llama_index.readers import BeautifulSoupWebReader
from llama_index import (
    SimpleDirectoryReader,
    GPTVectorStoreIndex,
    VectorStoreIndex,
    ServiceContext,
    download_loader,
    PromptHelper,
    StorageContext,
    load_index_from_storage,
    StorageContext,
    KnowledgeGraphIndex,
    load_graph_from_storage
)
from llama_index.query_engine import KnowledgeGraphQueryEngine
from llama_index.graph_stores import Neo4jGraphStore, NebulaGraphStore
from llama_index.embeddings import HuggingFaceEmbedding, OpenAIEmbedding, LangchainEmbedding
from llama_index.response.notebook_utils import display_response
from llama_index.graph_stores import SimpleGraphStore
from llama_index.vector_stores import ChromaVectorStore

In [6]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
from llama_index.llms import LlamaCPP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 58.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.27-cp310-cp310-manylinux_2_35_x86_64.whl size=8182245 sha256=80f2c5eedc25573fa6c41f8079c4b6c12771d8c67546c77c36cfad8fdc20e2cd
  Stored in directory: /root/.cache/pip/wheels/8c/92/37/ada3fcfdf537bab790219920443164923e6cbfcbd80174af23
Successfully built llama-cpp-python


In [7]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import HuggingFaceLLM
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf?download=true"

# Change LLMs and see different !!!

llm = LlamaCPP(model_url=model_url, model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    model_kwargs={"n_gpu_layers": 60},
    callback_manager = callback_manager,
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose = True)

total size (MB): 5131.41


4894it [00:32, 149.52it/s]                          
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
model_url = "https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q5_K_M.gguf?download=true"

# Change LLMs and see different !!!

llm_phi = LlamaCPP(model_url=model_url, model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    model_kwargs={"n_gpu_layers": 60},
    callback_manager = callback_manager,
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose = True)

total size (MB): 2072.68


1977it [00:15, 129.49it/s]
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path('/content/Employment-Manual-January-2023-trang-1.pdf'))

In [10]:
prompt = """
<s><INST> ## 1. Overview
You are a top-tier algorithm designed for extracting information about a guide for employees who want to understand labor laws in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly.Do not contain any explanations or appologies in your response.Non-compliance will result in termination.

Example:
 "Some text is provided below. Given the text, extract up to "
    "{max_knowledge_triplets} "
    "knowledge triplets in the form of (subject, predicate, object). Avoid stopwords.\n"
    "---------------------\n"
    "Example:"
    "Text: Alice is Bob's mother."
    "Triplets:\n(Alice, is mother of, Bob)\n"
    "Text: Philz is a coffee shop founded in Berkeley in 1982.\n"
    "Triplets:\n"
    "(Philz, is, coffee shop)\n"
    "(Philz, founded in, Berkeley)\n"
    "(Philz, founded in, 1982)\n"
    "---------------------\n"
    "Text: {text}\n"
    "Triplets:\n"
 </INST>
"""
prompt_template = PromptTemplate(prompt, prompt_type = "knowledge_triplet_extract")

In [ ]:
#embed_model = OpenAIEmbedding() #sentence-transformers/all-roberta-large-v1
embed_model = LangchainEmbedding(HuggingFaceEmbedding("sentence-transformers/all-roberta-large-v1")) #local:BAAI/bge-small-en-v1.5
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size = 128, chunk_overlap = 120)

In [12]:
service_context_phi = ServiceContext.from_defaults(llm=llm_phi, embed_model=embed_model, chunk_size = 128, chunk_overlap = 120)

In [13]:
username = "neo4j"
password = "7q0G7tP_ZxJGK-HH87pNoLMqcCO4sDN2cQnPIASwzLw"
url = "neo4j+s://39992655.databases.neo4j.io"
database = "neo4j"

In [14]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [16]:
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    #kg_triplet_extract_fn=extract_triplets,
    service_context=service_context,
    kg_triple_extract_template=prompt_template,
    #include_embedding = True,
    show_progress = True
)

Parsing nodes:   0%|          | 0/12 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/262 [00:00<?, ?it/s]

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [64]:
query_engine = index.as_query_engine(
    include_text=False,
    response_mode="tree_summarize", #test response !!!
    #embedding_mode="hybrid", #
    similarity_top_k=5,
    #explore_global_knowledge=True
)

In [85]:
query_str = "According to labor code tell me about DOLISA ?"
response = query_engine.query(query_str)

Llama.generate: prefix-match hit


Index was not constructed with embeddings, skipping embedding usage...


Llama.generate: prefix-match hit


In [ ]:
response

In [87]:
print(response)

 DOLISA, which stands for "Department of Labor, Invalids and Social Affairs," is an organization that is responsible for various tasks related to labor and employment issues in Vietnam. It is a subset of the Ministry of Labor, Invalids and Social Affairs (MOLISA).

According to the given information, DOLISA can confirm work permit exemptions for employers and expatriates. Employers must file an application with DOLISA to confirm work permit exemptions for their expatriate employees. Additionally, expatriates must meet certain conditions, such as being in good health, of full age (18 years or older), not being subject to prosecution, and working as a manager/general director or chief representative, expert, or technician, among other things.

Furthermore, DOLISA is the organization that issues legal documents related to labor and employment. It also ensures compliance with national policies on employment and provides guidance concerning these policies.

It's important to note that while

In [ ]:
!pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.4 MB/s eta 0:00:00


In [ ]:
from pyvis.network import Network
from IPython.display import HTML, display

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("example.html")

display(HTML(filename="example.html"))

In [ ]:
from IPython.display import IFrame

net.show("example.html")
display(IFrame("example.html", width=800, height=600))

example.html


In [17]:
index.storage_context.persist(persist_dir = './content/graph_chatbot') #save index

In [20]:
!zip -r /content/content/graph_chatbot.zip /content/content/graph_chatbot

  adding: content/content/graph_chatbot/ (stored 0%)
  adding: content/content/graph_chatbot/default__vector_store.json (deflated 19%)
  adding: content/content/graph_chatbot/index_store.json (deflated 84%)
  adding: content/content/graph_chatbot/docstore.json (deflated 88%)
  adding: content/content/graph_chatbot/image__vector_store.json (deflated 19%)


In [ ]:
%rm -rf content

In [33]:
storage_graph = StorageContext.from_defaults(persist_dir = './content/graph_chatbot',graph_store = graph_store)

In [44]:
kg_index = load_index_from_storage(
    storage_context = storage_graph,
    service_context = service_context,
    max_triplets_per_chunk = 10,
    llm = llm
)

In [45]:
kg_query_engine = kg_index.as_query_engine(
    include_text=False,
    response_mode="tree_summarize", #test response !!!
    #embedding_mode="hybrid", #
    similarity_top_k=5,
)

In [88]:
query_str = "According to labor code tell me about DOLISA"
response = kg_query_engine.query(query_str)

Llama.generate: prefix-match hit


Index was not constructed with embeddings, skipping embedding usage...


Llama.generate: prefix-match hit


In [89]:
print(response)

 Based on the provided information, DOLISA (Directorate of Labor, Invalids and Social Affairs) is an organization that is a subset of MOLISA (Ministry of Labor, Invalids and Social Affairs). It can issue legal documents related to labor and employment. Employers must file an application with DOLISA for work permit exemptions for expatriates, and DOLISA confirms the exemption if the employer meets certain conditions. The document required for a work permit application includes various information about the expatriate and the employment.


In [90]:
query_engine_load = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm_phi,
    verbose=True,
)

In [91]:
from llama_index.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
# chat_engine = ReActAgent.from_tools(
#     query_engine_tools, llm=llm, memory=memory, verbose=True
# )

chat_engine = kg_index.as_chat_engine(
    chat_mode="react",
    memory=memory,
    verbose=True,
)


In [ ]:
chat_engine.chat("According to labor code tell me about DOLISA")

## Compare and Combining KG Index and VectorStore Index

In [ ]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KGTableRetriever,
)

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes